In [2]:
import os
import kagglehub

## Importing data

In [3]:
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'face-mask-dataset' dataset.
Path to dataset files: /kaggle/input/face-mask-dataset


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
with_mask_path = '/kaggle/input/face-mask-dataset/data/with_mask'
without_mask_path = '/kaggle/input/face-mask-dataset/data/without_mask'

In [6]:
with_mask_images = [os.path.join(with_mask_path, i) for i in os.listdir(with_mask_path)]
without_mask_images = [os.path.join(without_mask_path, i) for i in os.listdir(without_mask_path)]

In [7]:
with_mask_labels = [1] * len(with_mask_images)
without_mask_labels = [0] * len(without_mask_images)

In [8]:
images = with_mask_images + without_mask_images
labels = with_mask_labels + without_mask_labels
df = pd.DataFrame({'files': images, 'class': labels})
df['class'] = df['class'].astype(str)

In [9]:
df_train, df_temp = train_test_split(df, test_size=0.3, stratify=df['class'], random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, stratify=df_temp['class'], random_state=42)

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.25,
    brightness_range=[0.7,1.3],
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

Found 5287 validated image filenames belonging to 2 classes.


In [12]:
val_gen = test_val_datagen.flow_from_dataframe(
    dataframe=df_val,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)


Found 1133 validated image filenames belonging to 2 classes.


In [13]:
test_gen = test_val_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)


Found 1133 validated image filenames belonging to 2 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense , Dropout, BatchNormalization , Activation
from tensorflow.keras.regularizers import l2

In [15]:
model = Sequential()

In [16]:
model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(1e-4), padding='same', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.add(Conv2D(64, (3, 3), kernel_regularizer=l2(1e-4), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

In [18]:
model.add(Conv2D(128, (3, 3), kernel_regularizer=l2(1e-4), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

In [19]:
model.add(Conv2D(256, (3, 3), kernel_regularizer=l2(1e-4), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

In [20]:
model.add(Flatten())
model.add(Dense(256, kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [21]:
model.add(Dense(128, kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [22]:
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
from keras.callbacks  import EarlyStopping

In [25]:
earlystop = EarlyStopping(patience=2 , restore_best_weights=True)

In [26]:
model.fit(train_gen, validation_data=val_gen, epochs=30 ,  callbacks=[earlystop])

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
133/166 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.7092 - loss: 0.6927

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


166/166 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.7224 - loss: 0.6759 - val_accuracy: 0.5093 - val_loss: 1.6050
Epoch 2/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.8475 - loss: 0.5095 - val_accuracy: 0.6540 - val_loss: 0.8598
Epoch 3/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.8579 - loss: 0.4873 - val_accuracy: 0.7855 - val_loss: 0.6762
Epoch 4/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 274s 2s/step - accuracy: 0.8616 - loss: 0.4784 - val_accuracy: 0.8773 - val_loss: 0.4169
Epoch 5/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 280s 2s/step - accuracy: 0.8818 - loss: 0.4465 - val_accuracy: 0.9073 - val_loss: 0.3936
Epoch 6/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 268s 2s/step - accuracy: 0.8886 - loss: 0.4176 - val_accuracy: 0.8703 - val_loss: 0.5035
Epoch 7/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 275s 2s/step - accuracy: 0.9050 - loss: 0.4046 - val_accuracy: 0.9479 - val_loss: 0.3139
Epoch 8/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.9097 - loss: 0.3878 - val_accuracy: 0.946

In [27]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc*100:.2f}%")

36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 359ms/step - accuracy: 0.9587 - loss: 0.2691
Test Accuracy: 95.94%


In [28]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_gen)
y_pred_classes = (y_pred >= 0.5).astype(int)

y_true = test_gen.classes

print(classification_report(y_true, y_pred_classes, target_names=["Without Mask", "With Mask"]))

36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 322ms/step
              precision    recall  f1-score   support

Without Mask       0.95      0.98      0.96       574
   With Mask       0.97      0.94      0.96       559

    accuracy                           0.96      1133
   macro avg       0.96      0.96      0.96      1133
weighted avg       0.96      0.96      0.96      1133



In [29]:
model.save('face_mask_detection3.h5')